In [1]:
from reader_utils.corpus_builder_utils import *
from note_utils.pitch_dictionary import PitchDictionary
from reader_utils.dataset_builder import DatasetBuilder
from model_utils.model_utils import *
import io

In [2]:
pd = PitchDictionary("dataset_objects/pitches_dict.txt")

vocab_size = pd.get_vocabulary_size()
window_size = 3
num_ns = 10
embedding_layer_name = "n2v_embedding"

In [3]:
db = DatasetBuilder("dataset_objects/full_corpus")

In [4]:
# takes around 66 mins for window_size = 3, num_ns = 5
dataset = db.build_word_to_vec_dataset(
        vocab_size, window_size, num_ns, skip_amount=1)

100%|██████████| 2200/2200 [23:04<00:00,  1.59it/s]


In [14]:
embedding_dim = 16

n2v = build_note_to_vec_model(
        vocab_size, embedding_dim, num_ns, embedding_layer_name)

In [15]:
# takes around 11 mins for embedding_dim = 32
epochs = 20
n2v.fit(dataset, epochs=epochs)

Epoch 1/20
5136/5136 [==============================] - 19s 4ms/step - loss: 0.6175 - accuracy: 0.8060
Epoch 2/20
5136/5136 [==============================] - 18s 4ms/step - loss: 0.4785 - accuracy: 0.8343
Epoch 3/20
5136/5136 [==============================] - 18s 4ms/step - loss: 0.4695 - accuracy: 0.8372
Epoch 4/20
5136/5136 [==============================] - 16s 3ms/step - loss: 0.4676 - accuracy: 0.8378
Epoch 5/20
5136/5136 [==============================] - 17s 3ms/step - loss: 0.4666 - accuracy: 0.8382
Epoch 6/20
5136/5136 [==============================] - 17s 3ms/step - loss: 0.4658 - accuracy: 0.8384
Epoch 7/20
5136/5136 [==============================] - 17s 3ms/step - loss: 0.4655 - accuracy: 0.8385
Epoch 8/20
5136/5136 [==============================] - 19s 4ms/step - loss: 0.4654 - accuracy: 0.8386
Epoch 9/20
5136/5136 [==============================] - 17s 3ms/step - loss: 0.4654 - accuracy: 0.8386
Epoch 10/20
5136/5136 [==============================] - 16s 3ms/step - l

In [16]:
vocab = pd.get_vocabulary()
weights = n2v.get_layer(embedding_layer_name).get_weights()[0]

In [17]:
file_suffix = "ws" + str(window_size) + "_ns" + str(num_ns) + \
        "_ed" + str(embedding_dim) + "_ep" + str(epochs)
vactors_file_name = "vectors_" + file_suffix + ".tsv"
metadata_file_name = "metadata_" + file_suffix + ".tsv"

vectors_file = io.open(vactors_file_name, 'w', encoding='utf-8')
metadata_file = io.open(metadata_file_name, 'w', encoding='utf-8')

for index, word in enumerate(vocab):
    if index in [0, 1]: # <unk> and es
        continue
    vec = weights[index]
    vectors_file.write('\t'.join([str(x) for x in vec]) + "\n")
    metadata_file.write(word + "\n")

vectors_file.close()
metadata_file.close()